# Single Table Modeling

**SDV** supports modeling single table datasets. It provides unique features for making it easy for the user 
to learn models and synthesize datasets. Some important features of sdv.tables include:

* Support for tables with primarykey
* Support to anonymize certain fields like addresses, emails, phone numbers, names and other PII information. 
  We use faker library for this. The full list of categories supported corresponds to the `Faker` library 
  [provider names](https://faker.readthedocs.io/en/master/providers.html).
* Support for a number of different data types - categorical, numerical, discrete-ordinal and datetimes.
* Support multiple types of statistical and deep learning models:
  * GaussianCopula: A tool to model multivariate distributions using [copula functions](
    https://en.wikipedia.org/wiki/Copula_%28probability_theory%29). Based on our [Copulas Library](
    https://github.com/sdv-dev/Copulas).
  * CTGAN: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high 
    fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

<div class="alert alert-info">
    <b>Note:</b> We are adding a number of additional features and functionality to make it easy to model single table datasets. For example, we are adding ways for users to add inter-column constraints . If you find a unique use case that we do not support consider suggesting and adding examples here.
</div>

## Quick usage

Let's consider a dataset from our demo datasets. 

In [1]:
import warnings
warnings.simplefilter('ignore')

from sdv import load_demo

users = load_demo()['users']

This will return a table with 4 fields:

* `user_id`: A unique identifier of the user.
* `country`: A 2 letter code of the country of residence of the user.
* `gender`: A single letter code, `M` or `F`, indicating the user gender. Note that this demo simulates the case where some users did not indicate the gender, which resulted in empty data values in some rows.
* `age`: The age of the user, in years.

In [2]:
users

,user_id,country,gender,age
0,0,US,M,34
1,1,UK,F,23
2,2,ES,None,44
3,3,UK,M,22
4,4,US,F,54
5,5,DE,M,57
6,6,BG,F,45
7,7,ES,None,41
8,8,FR,F,23
9,9,UK,None,30


We notice that there are some additional properties in this dataset:

* First, `user_id` field in our table is the `primary_key` and each row has a `unique` value, so we do not
  want our model to attempt to learn it.
* Second, let's say we want to `anonymize` the countries of residence of our `users`, to avoid disclosing
  such information. 
* Third, we notice that there is missing data for the `gender` column. 

Let us use the `GaussianCopula` to model this data and then sample synthetic data from the model. In order
to properly model our data we will need to provide some additional information to our model. Once we have
prepared the arguments for our model we are ready to import it, create an instance and fit it to our data.

In [3]:
from sdv.tabular import GaussianCopula

model = GaussianCopula(
    primary_key='user_id',
    anonymize_fields={'country':'country_code'}
)
model.fit(users)

2020-07-21 17:38:16,265 - INFO - table - Loading transformer OneHotEncodingTransformer for field country
2020-07-21 17:38:16,266 - INFO - table - Loading transformer OneHotEncodingTransformer for field gender
2020-07-21 17:38:16,266 - INFO - table - Loading transformer NumericalTransformer for field age
2020-07-21 17:38:16,279 - INFO - gaussian - Fitting GaussianMultivariate()


**Notice** that the model `fitting` process took care of transforming the different fields using the
appropriate [Reversible Data Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has
a format that the GaussianMultivariate model from the [copulas](https://github.com/sdv-dev/Copulas)
library can handle.

## Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from your model.

In [4]:
sampled = model.sample(5)

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [5]:
sampled

,user_id,country,gender,age
0,0,BF,M,48
1,1,BF,F,31
2,2,ES,F,37
3,3,SI,M,54
4,4,PT,M,23


<div class="alert alert-info">
    <b>Note:</b>You can control the number of rows by specifying the number of <tt>samples</tt> in the
    <tt>model.sample(<n_samples>)</tt>. To test, try <tt>model.sample(10000)</tt>. Note that the original 
        table only had 10 rows.
</div>

## Let's consider using CTGAN

In this second part of the tutorial we will be using the CTGAN model to learn the data from the
demo dataset called `census`, which is based on the [UCI Adult Census Dataset]('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data').

In [6]:
from sdv import load_demo

census = load_demo('census')['census']

2020-07-21 17:38:17,934 - INFO - __init__ - Loading table census


This will return a table with several rows of multiple data types:

In [7]:
census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In this case there is no `primary_key` to setup and we will not be `anonymizing` anything, so the
only thing that we will pass to the `CTGAN` model is the `number of epochs` that we want it to
perform when it learns the data, which we will keep low to make this execution quick.

In [8]:
from sdv.tabular import CTGAN

model = CTGAN(epochs=10)

Once the instance is created, we can fit it to our data. 

<div class="alert alert-info">
    Note that this process might take some time to finish, especially on non-GPU enabled systems,
    so in this case we will be passing only a <tt>subsample</tt> of the data to accelerate the process.
</div>

In [9]:
model.fit(census.sample(1000))

2020-07-21 17:38:18,308 - INFO - table - Loading transformer NumericalTransformer for field age
2020-07-21 17:38:18,308 - INFO - table - Loading transformer LabelEncodingTransformer for field workclass
2020-07-21 17:38:18,309 - INFO - table - Loading transformer NumericalTransformer for field fnlwgt
2020-07-21 17:38:18,309 - INFO - table - Loading transformer LabelEncodingTransformer for field education
2020-07-21 17:38:18,310 - INFO - table - Loading transformer NumericalTransformer for field education-num
2020-07-21 17:38:18,310 - INFO - table - Loading transformer LabelEncodingTransformer for field marital-status
2020-07-21 17:38:18,311 - INFO - table - Loading transformer LabelEncodingTransformer for field occupation
2020-07-21 17:38:18,311 - INFO - table - Loading transformer LabelEncodingTransformer for field relationship
2020-07-21 17:38:18,311 - INFO - table - Loading transformer LabelEncodingTransformer for field race
2020-07-21 17:38:18,312 - INFO - table - Loading transforme

Epoch 1, Loss G: 2.0458, Loss D: -0.0274
Epoch 2, Loss G: 1.9645, Loss D: -0.0890
Epoch 3, Loss G: 1.9711, Loss D: -0.1531
Epoch 4, Loss G: 1.9269, Loss D: -0.2453
Epoch 5, Loss G: 1.8742, Loss D: -0.2927
Epoch 6, Loss G: 1.8715, Loss D: -0.3950
Epoch 7, Loss G: 1.8721, Loss D: -0.5831
Epoch 8, Loss G: 1.8332, Loss D: -0.7313
Epoch 9, Loss G: 1.7187, Loss D: -0.9058
Epoch 10, Loss G: 1.6420, Loss D: -0.8540


### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from our model just like we did with the `GaussianCopula` model.

In [10]:
sampled = model.sample(1000)

This will return a table identical to the one which the model was fitted on, but filled with `synthetic` data
which resembles the original one.

In [11]:
sampled.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,28,?,132148,HS-grad,8,Divorced,Prof-specialty,Not-in-family,White,Female,12633,-22,36,Italy,<=50K
1,15,Private,108655,Some-college,12,Never-married,?,Not-in-family,White,Male,-134,-14,56,Nicaragua,<=50K
2,21,Private,177471,HS-grad,11,Married-civ-spouse,Transport-moving,Unmarried,White,Male,8657,6,39,United-States,>50K
3,53,Private,58238,9th,11,Widowed,Craft-repair,Unmarried,Other,Male,357,0,33,Puerto-Rico,<=50K
4,20,Local-gov,199353,10th,3,Separated,?,Own-child,Other,Female,373,11,56,Trinadad&Tobago,<=50K
5,33,Federal-gov,61574,5th-6th,9,Divorced,Prof-specialty,Husband,Black,Male,-103,21,42,Dominican-Republic,<=50K
6,22,Private,256859,5th-6th,4,Married-civ-spouse,Sales,Not-in-family,White,Female,387,-8,21,India,<=50K
7,57,?,-1058,7th-8th,3,Married-civ-spouse,Other-service,Own-child,White,Male,883,26,29,Portugal,<=50K
8,19,Local-gov,99863,Some-college,3,Widowed,Exec-managerial,Husband,Asian-Pac-Islander,Male,-189,-7,40,Portugal,<=50K
9,15,Private,95500,11th,1,Never-married,Adm-clerical,Not-in-family,Amer-Indian-Eskimo,Female,292,2055,50,Philippines,<=50K


## Frequently encountered needs

### How can I evaluate the quality of my synthetic data?

In some cases, you will want to know how similar the generated is to the original one.

For this you can use the `evaluation` framework included in SDV by simply importing the
`sdv.evaluation.evaluate` function and calling it passing it both the synthetic and the
real data.

In [12]:
from sdv.evaluation import evaluate

evaluate(sampled, census)

-133.19682079258746